# **Plot and Stats for Tidy Format**
---

<font size = 4>Colab Notebook for Plotting data


<font size = 4>Notebook created by [Guillaume Jacquemet](https://cellmig.org/)



# **Part 0. Before getting started**
---

<font size = 5>**Important notes**

---
## Data Requirements for Analysis

<font size = 4>For a successful analysis using this notebook, ensure your data meets the following criteria:

## Notebook Data Format and Requirements Documentation

This document details the prerequisites for data to be analyzed effectively within this notebook. Ensuring adherence to these guidelines will facilitate accurate and efficient data analysis.

### File Format
- **CSV**: Data should be in CSV (Comma-Separated Values) format, easily generated from spreadsheet applications (e.g., Excel, Google Sheets) or statistical software (e.g., R, Python).
- **Copy and Paste**: Data can be directly copied and pasted from a spreedsheet software.

### Data Structure: Tidy Format
Data must follow the tidy data principles for optimal processing:
- **Each Variable Forms a Column**: Every column represents a single variable.
- **Each Observation Forms a Row**: Every row represents a single observation.
- **Each Type of Observational Unit Forms a Table**: Different observational units should be in separate tables or clearly distinguishable.

### Essential Columns
Your dataset must include specific columns for analysis:
- **Biological Repeat Column**: Identifies biological replicates. Names can vary (e.g., "Repeat", "Bio_Replicate") but must consistently identify each biological repeat.
- **Condition Column**: Categorizes observations by experimental conditions or treatments. Names can vary (e.g., "Condition", "Treatment") but must provide clear, consistent categorization.

### Data Preparation Tips
- **Consistency and Clarity**: Ensure consistent and descriptive naming within "Biological Repeat" and "Condition" columns.
- **Data Cleaning**: Address missing or erroneous entries in these essential columns to prevent analysis issues.

### Column Naming Flexibility
- The exact names of the "Biological Repeat" and "Condition" columns are flexible to fit various dataset structures and terminologies. You'll specify these columns when using the notebook.

Adhering to these guidelines ensures your data is primed for the notebook's analytical capabilities, allowing for insightful comparisons across biological repeats and conditions.




In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

--------------------------------------------------------
# **Part 1. Prepare the session and load your data**
--------------------------------------------------------


## **1.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install
!pip -q install pandas scikit-learn
!pip -q install plotly
!pip -q install prettytable
!pip -q install pmoss



In [ ]:
#@markdown ##Play to load the dependancies

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import itertools
from matplotlib.gridspec import GridSpec
import requests

!pip freeze > requirements.txt


# Function to calculate Cohen's d
def cohen_d(group1, group2):
    diff = group1.mean() - group2.mean()
    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()
    pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
    d = diff / np.sqrt(pooled_var)
    return d

def save_dataframe_with_progress(df, path, desc="Saving", chunk_size=50000):
    """Save a DataFrame with a progress bar."""

    # Estimating the number of chunks based on the provided chunk size
    num_chunks = int(len(df) / chunk_size) + 1

    # Create a tqdm instance for progress tracking
    with tqdm(total=len(df), unit="rows", desc=desc) as pbar:
        # Open the file for writing
        with open(path, "w") as f:
            # Write the header once at the beginning
            df.head(0).to_csv(f, index=False)

            for chunk in np.array_split(df, num_chunks):
                chunk.to_csv(f, mode="a", header=False, index=False)
                pbar.update(len(chunk))

def check_for_nans(df, df_name):
    """
    Checks the given DataFrame for NaN values and prints the count for each column containing NaNs.

    Args:
    df (pd.DataFrame): DataFrame to be checked for NaN values.
    df_name (str): The name of the DataFrame as a string, used for printing.
    """
    # Check if the DataFrame has any NaN values and print a warning if it does.
    nan_columns = df.columns[df.isna().any()].tolist()

    if nan_columns:
        for col in nan_columns:
            nan_count = df[col].isna().sum()
            print(f"Column '{col}' in {df_name} contains {nan_count} NaN values.")
    else:
        print(f"No NaN values found in {df_name}.")


import pandas as pd
import os

def save_parameters(params, file_path, param_type):
    # Convert params dictionary to a DataFrame for human readability
    new_params_df = pd.DataFrame(list(params.items()), columns=['Parameter', 'Value'])
    new_params_df['Type'] = param_type

    if os.path.exists(file_path):
        # Read existing file
        existing_params_df = pd.read_csv(file_path)

        # Merge the new parameters with the existing ones
        # Update existing parameters or append new ones
        updated_params_df = pd.merge(existing_params_df, new_params_df,
                                     on=['Type', 'Parameter'],
                                     how='outer',
                                     suffixes=('', '_new'))

        # If there's a new value, update it, otherwise keep the old value
        updated_params_df['Value'] = updated_params_df['Value_new'].combine_first(updated_params_df['Value'])

        # Drop the temporary new value column
        updated_params_df.drop(columns='Value_new', inplace=True)
    else:
        # Use new parameters DataFrame directly if file doesn't exist
        updated_params_df = new_params_df

    # Save the updated DataFrame to CSV
    updated_params_df.to_csv(file_path, index=False)


## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/content/gdrive')
%cd /gdrive



## **1.3. Load your dataset**
---

<font size = 4> Please ensure that your data is properly organised (see above)


In [ ]:
#@markdown ##Load your dataset:


import pandas as pd
import os
from io import StringIO
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize dataset_df as an empty DataFrame globally
dataset_df = pd.DataFrame()


# Create widgets
dataset_path_input = widgets.Text(
    value='',
    placeholder='Enter the path to your dataset',
    description='Dataset Path:',
    layout={'width': '100%'}
)

results_folder_input = widgets.Text(
    value='',
    placeholder='Enter the path to your results folder',
    description='Results Folder:',
    layout={'width': '100%'}
)

data_textarea = widgets.Textarea(
    value='',
    placeholder='Or copy and paste your tab sperated data here (direct copy and paste from a spreedsheet)',
    description='Or Paste Data:',
    layout={'width': '100%', 'height': '200px'}
)

load_button = widgets.Button(
    description='Load Data',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to load the data',
)

output = widgets.Output()

# Load data function
def load_data(b):
    global dataset_df
    global Results_Folder

    with output:
        clear_output()
        Results_Folder = results_folder_input.value.strip()
        if not Results_Folder:
            Results_Folder = './Results'  # Default path if not provided
        if not os.path.exists(Results_Folder):
            os.makedirs(Results_Folder)  # Create the folder if it doesn't exist
        print(f"Results folder is located at: {Results_Folder}")

        if dataset_path_input.value.strip():
            dataset_path = dataset_path_input.value.strip()
            try:
                dataset_df = pd.read_csv(dataset_path)
                print(f"Loaded dataset from {dataset_path}")
            except Exception as e:
                print(f"Failed to load dataset from {dataset_path}: {e}")
        elif data_textarea.value.strip():
            input_data = StringIO(data_textarea.value)
            try:
                dataset_df = pd.read_csv(input_data, sep='\t')
                print("Loaded dataset from pasted tab-separated data")
            except Exception as e:
                print(f"Failed to load dataset from pasted data: {e}")
        else:
            print("No dataset path provided or data pasted. Please provide a dataset.")
            return

        # Perform a check for NaNs or any other required processing here
        check_for_nans(dataset_df, "your dataset")

        display(dataset_df.head())

# Set the button click event
load_button.on_click(load_data)

# Display the widgets
display(widgets.VBox([dataset_path_input, results_folder_input, data_textarea, load_button, output]))


## **1.4. Map your data**
---

## Required Columns

<font size = 4>To plot your data, we need to ensure the presence of specific columns in the dataset. Here's a breakdown of the required columns:

- **`Condition`**: Identifies the biological condition.

- **`Repeat`**: Represents the biological repeat.

- **`Variable`**: Represents the variable to plot.



In [ ]:
#@markdown ##Map your dataset:


import ipywidgets as widgets  # Ensure we have the required widgets module imported
import pandas as pd

def single_stage_column_mapping(df):
    # Define the columns we need to map: Condition, Repeat
    mappings = {
        'Condition': 'Identifies the biological conditions.',
        'Repeat': 'Represents the biological repeats.'
    }

    dropdowns = {}
    for key, description in mappings.items():
        description_label = widgets.Label(f"{key} ({description}):")
        dropdowns[key] = widgets.Dropdown(options=df.columns, layout=widgets.Layout(width='250px'))

        # Use HBox to display the description label next to the dropdown
        hbox = widgets.HBox([description_label, dropdowns[key]])
        display(hbox)

    confirm_button = widgets.Button(description="Confirm Mappings")

    def confirm_mappings(button):
        # Perform the mapping based on the user selection
        column_mapping = {dropdown.value: key for key, dropdown in dropdowns.items()}
        new_df = df.rename(columns=column_mapping)

        print("Columns Mapped Successfully!")

        # Count and print unique conditions
        unique_conditions = new_df['Condition'].unique()
        print(f"Number of unique conditions: {len(unique_conditions)}")
        print("Conditions:", ", ".join(unique_conditions))

        # Count and print biological repeats
        unique_repeats = new_df['Repeat'].unique()
        print(f"Number of biological repeats: {len(unique_repeats)}")
        print("Repeats:", ", ".join(map(str, unique_repeats)))


        # Check that each biological condition has exactly the same repeat names
        condition_repeats = new_df.groupby('Condition')['Repeat'].apply(set)
        if len(set(map(frozenset, condition_repeats))) == 1:
            print("All biological conditions have exactly the same repeat names.")
        else:
            print("Warning: Not all biological conditions have the same repeat names.")

        # Update the global dataset_df with the new mappings
        global dataset_df
        dataset_df = new_df

    confirm_button.on_click(confirm_mappings)
    display(confirm_button)

single_stage_column_mapping(dataset_df)


-------------------------------------------

# **Part 2. Plot your entire dataset**
-------------------------------------------

<font size = 4> In this section you can plot your data. Data and graphs are automatically saved in your result folder.


## **2.1. Plot your entire dataset**
--------

##**Statistical analyses**
### Cohen's d (Effect Size):
<font size = 4>Cohen's d measures the size of the difference between two groups, normalized by their pooled standard deviation. Values can be interpreted as small (0 to 0.2), medium (0.2 to 0.5), or large (0.5 and above) effects. It helps quantify how significant the observed difference is, beyond just being statistically significant.

### Randomization Test:
<font size = 4>This non-parametric test evaluates if observed differences between conditions could have arisen by random chance. It shuffles condition labels multiple times, recalculating the Cohen's d each time. The resulting p-value, which indicates the likelihood of observing the actual difference by chance, provides evidence against the null hypothesis: a smaller p-value implies stronger evidence against the null.

### Bonferroni Correction:
<font size = 4>Given multiple comparisons, the Bonferroni Correction adjusts significance thresholds to mitigate the risk of false positives. By dividing the standard significance level (alpha) by the number of tests, it ensures that only robust findings are considered significant. However, it's worth noting that this method can be conservative, sometimes overlooking genuine effects.

In [ ]:
# @title ##Plot (entire dataset)

import ipywidgets as widgets
from ipywidgets import Layout, VBox, Button, Accordion, SelectMultiple, IntText
import pandas as pd
import os
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FixedLocator



# Parameters to adapt in function of the notebook section
base_folder = f"{Results_Folder}/Plots"
Conditions = 'Condition'
df_to_plot = dataset_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def get_selectable_columns(df):
    # Exclude certain columns from being plotted
    exclude_cols = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']
    # Select only numerical columns
    return [col for col in df.columns if (df[col].dtype.kind in 'biufc') and (col not in exclude_cols)]


def display_variable_checkboxes(selectable_columns):
    # Create checkboxes for selectable columns
    variable_checkboxes = [widgets.Checkbox(value=False, description=col) for col in selectable_columns]

    # Display checkboxes in the notebook
    display(widgets.VBox([
        widgets.Label('Variables to Plot:'),
        widgets.GridBox(variable_checkboxes, layout=widgets.Layout(grid_template_columns="repeat(%d, 300px)" % 3)),
    ]))
    return variable_checkboxes

def create_condition_selector(df, column_name):
    conditions = df[column_name].unique()
    condition_selector = SelectMultiple(
        options=conditions,
        description='Conditions:',
        disabled=False,
        layout=Layout(width='100%')  # Adjusting the layout width
    )
    return condition_selector

def display_condition_selection(df, column_name):
    condition_selector = create_condition_selector(df, column_name)

    condition_accordion = Accordion(children=[VBox([condition_selector])])
    condition_accordion.set_title(0, 'Select Conditions')
    display(condition_accordion)
    return condition_selector


def plot_selected_vars(button, variable_checkboxes, df, Conditions, Results_Folder, condition_selector):

    plt.clf()  # Clear the current figure before creating a new plot
    print("Plotting in progress...")

  # Get selected variables
    variables_to_plot = [box.description for box in variable_checkboxes if box.value]
    n_plots = len(variables_to_plot)

    if n_plots == 0:
        print("No variables selected for plotting")
        return

  # Get selected conditions
    selected_conditions = condition_selector.value
    n_selected_conditions = len(selected_conditions)

    if n_selected_conditions == 0:
        print("No conditions selected for plotting")
        return

# Use only selected and ordered conditions
    filtered_df = df[df[Conditions].isin(selected_conditions)].copy()

# Initialize matrices to store effect sizes and p-values for each variable
    effect_size_matrices = {}
    p_value_matrices = {}
    bonferroni_matrices = {}

    unique_conditions = filtered_df[Conditions].unique().tolist()
    num_comparisons = len(unique_conditions) * (len(unique_conditions) - 1) // 2
    alpha = 0.05
    corrected_alpha = alpha / num_comparisons
    n_iterations = 1000

# Loop through each variable to plot
    for var in variables_to_plot:

      pdf_pages = PdfPages(f"{Results_Folder}/pdf/{var}_Boxplots_and_Statistics.pdf")
      effect_size_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      p_value_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      bonferroni_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)

      for cond1, cond2 in itertools.combinations(unique_conditions, 2):
        group1 = df[df[Conditions] == cond1][var]
        group2 = df[df[Conditions] == cond2][var]

        original_d = abs(cohen_d(group1, group2))
        effect_size_matrix.loc[cond1, cond2] = original_d
        effect_size_matrix.loc[cond2, cond1] = original_d  # Mirroring

        count_extreme = 0
        for i in range(n_iterations):
            combined = pd.concat([group1, group2])
            shuffled = combined.sample(frac=1, replace=False).reset_index(drop=True)
            new_group1 = shuffled[:len(group1)]
            new_group2 = shuffled[len(group1):]

            new_d = abs(cohen_d(new_group1, new_group2))
            if np.abs(new_d) >= np.abs(original_d):
                count_extreme += 1

        p_value = count_extreme / n_iterations
        p_value_matrix.loc[cond1, cond2] = p_value
        p_value_matrix.loc[cond2, cond1] = p_value  # Mirroring

        # Apply Bonferroni correction
        bonferroni_corrected_p_value = min(p_value * num_comparisons, 1.0)
        bonferroni_matrix.loc[cond1, cond2] = bonferroni_corrected_p_value
        bonferroni_matrix.loc[cond2, cond1] = bonferroni_corrected_p_value  # Mirroring

      effect_size_matrices[var] = effect_size_matrix
      p_value_matrices[var] = p_value_matrix
      bonferroni_matrices[var] = bonferroni_matrix

    # Concatenate the three matrices side-by-side
      combined_df = pd.concat(
        [
            effect_size_matrices[var].rename(columns={col: f"{col} (Effect Size)" for col in effect_size_matrices[var].columns}),
            p_value_matrices[var].rename(columns={col: f"{col} (P-Value)" for col in p_value_matrices[var].columns}),
            bonferroni_matrices[var].rename(columns={col: f"{col} (Bonferroni-corrected P-Value)" for col in bonferroni_matrices[var].columns})
        ], axis=1
    )

    # Save the combined DataFrame to a CSV file
      combined_df.to_csv(f"{Results_Folder}/csv/{var}_statistics_combined.csv")

    # Create a new figure
      fig = plt.figure(figsize=(16, 10))

    # Create a gridspec for 2 rows and 4 columns
      gs = GridSpec(2, 3, height_ratios=[1.5, 1])

    # Create the ax for boxplot using the gridspec
      ax_box = fig.add_subplot(gs[0, :])

    # Extract the data for this variable
      data_for_var = df[[Conditions, var, 'Repeat' ]]

    # Save the data_for_var to a CSV for replotting
      data_for_var.to_csv(f"{Results_Folder}/csv/{var}_boxplot_data.csv", index=False)

    # Calculate the Interquartile Range (IQR) using the 25th and 75th percentiles
      Q1 = df[var].quantile(0.25)
      Q3 = df[var].quantile(0.75)
      IQR = Q3 - Q1

    # Define bounds for the outliers
      multiplier = 10
      lower_bound = Q1 - multiplier * IQR
      upper_bound = Q3 + multiplier * IQR

    # Plotting
      sns.boxplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, color='lightgray')  # Boxplot
      sns.stripplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
      ax_box.set_ylim([max(min(filtered_df[var]), lower_bound), min(max(filtered_df[var]), upper_bound)])
      ax_box.set_title(f"{var}")
      ax_box.set_xlabel('Condition')
      ax_box.set_ylabel(var)
      tick_labels = ax_box.get_xticklabels()
      tick_locations = ax_box.get_xticks()
      ax_box.xaxis.set_major_locator(FixedLocator(tick_locations))
      ax_box.set_xticklabels(tick_labels, rotation=90)
      ax_box.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

    # Statistical Analyses and Heatmaps

    # Effect Size heatmap ax
      ax_d = fig.add_subplot(gs[1, 0])
      sns.heatmap(effect_size_matrices[var].fillna(0), annot=True, cmap="viridis", cbar=True, square=True, ax=ax_d, vmax=1)
      ax_d.set_title(f"Effect Size (Cohen's d) for {var}")

    # p-value heatmap ax
      ax_p = fig.add_subplot(gs[1, 1])
      sns.heatmap(p_value_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_p, vmax=0.1)
      ax_p.set_title(f"Randomization Test p-value for {var}")

    # Bonferroni corrected p-value heatmap ax
      ax_bonf = fig.add_subplot(gs[1, 2])
      sns.heatmap(bonferroni_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_bonf, vmax=0.1)
      ax_bonf.set_title(f"Bonferroni-corrected p-value for {var}")

      plt.tight_layout()
      pdf_pages.savefig(fig)

    # Close the PDF
      pdf_pages.close()

condition_selector = display_condition_selection(df_to_plot, Conditions)
selectable_columns = get_selectable_columns(df_to_plot)
variable_checkboxes = display_variable_checkboxes(selectable_columns)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, variable_checkboxes, df_to_plot, Conditions, base_folder, condition_selector))
display(button)

## **2.2. Export data summaries**
--------

In [ ]:
import pandas as pd
from prettytable import PrettyTable
import os

# @title ##Export the data summaries


# Assuming Results_Folder, dataset_df, and Conditions are defined
save_path = f"{Results_Folder}/variables_summary"
df_to_plot = dataset_df  # Assuming dataset_df is the DataFrame to work with

if not os.path.exists(save_path):
  os.makedirs(save_path)

def generate_display_and_save_statistics(df, columns, Conditions, save_path):
    """
    Generates, displays using prettytable, and saves as CSV the statistical summaries
    for selected columns of the DataFrame, grouped by the specified condition column.

    Parameters:
    - df: DataFrame to analyze.
    - columns: List of column names to generate statistics for.
    - Conditions: Column name to group by.
    - save_path: Directory path where CSV files will be saved.
    """
    # Ensure the save directory exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for var in columns:
        if var in df.columns:
            # Compute descriptive statistics and additional metrics
            grouped_stats = df.groupby(Conditions)[var].describe()
            variance = df.groupby(Conditions)[var].var().rename('variance')
            skewness = df.groupby(Conditions)[var].skew().rename('skewness')
            kurtosis = df.groupby(Conditions)[var].apply(pd.DataFrame.kurt).rename('kurtosis')

            # Concatenate all statistics into a single DataFrame
            all_stats = pd.concat([grouped_stats, variance, skewness, kurtosis], axis=1)

            # Save the summary to a CSV file
            csv_filename = f"{var}_summary.csv"
            all_stats.to_csv(os.path.join(save_path, csv_filename))
            print(f"Saved statistical summary for {var} to {csv_filename}")

            # Initialize PrettyTable, using the DataFrame's columns as table fields
            table = PrettyTable()
            table.field_names = ["Condition"] + list(all_stats.columns)

            # Populate the table with data
            for condition, row in all_stats.iterrows():
                table.add_row([condition] + row.tolist())

            # Set table alignment, style, etc.
            table.align = 'r'
            print(f"Statistical Summary for {var}:\n{table}")

generate_display_and_save_statistics(df_to_plot, selectable_columns, Conditions, save_path)


## **2.3. pMoSS (p-value Model using the Sample Size)** (To implement)
--------

--------
# **Part 3. Plot a balanced dataset (batch correction)**
--------

      



## **3.1. Assess if your dataset is balanced**
---

In data analyses, the balance of the dataset is important, particularly in ensuring that each biological repeat carries equal weight. Here's why this balance is essential:

### Accurate Representation of Biological Variability

- **Capturing True Biological Variation**: Biological repeats are crucial for capturing the natural variability inherent in biological systems. Equal weighting ensures that this variability is accurately represented.
- **Reducing Sampling Bias**: By balancing the dataset, we avoid overemphasizing the characteristics of any single repeat, which might not be representative of the broader biological context.

If your data is too imbalanced, it may be useful to ensure that this does not shift your results.



In [ ]:
# @title ##Check the number of values per condition per repeats

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

def count_values_by_condition_and_repeat(df, Results_Folder, condition_col='Condition', repeat_col='Repeat'):
    """
    Counts the number of rows for each combination of condition and repeat, plots a histogram,
    and adds the counts on top of each bar in the graph.
    """

    # Ensure the results folder exists
    Balanced_dataset_folder = os.path.join(Results_Folder, "Balanced_dataset")
    if not os.path.exists(Balanced_dataset_folder):
        os.makedirs(Balanced_dataset_folder)

    # Counting rows per condition and repeat
    value_counts = df.groupby([condition_col, repeat_col]).size().reset_index(name='Number_of_Values')

    # Preparing data for plotting
    pivot_df = value_counts.pivot(index=condition_col, columns=repeat_col, values='Number_of_Values').fillna(0)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = pivot_df.plot(kind='bar', stacked=True, ax=ax)
    ax.set_xlabel('Condition')
    ax.set_ylabel('Number of Values')
    ax.set_title('Stacked Histogram of Value Counts per Condition and Repeat')
    ax.legend(title=repeat_col)

    # Adding counts on top of each bar
    for bar in bars.patches:
        height = bar.get_height()
        if height > 0:  # Only annotate non-zero heights to avoid clutter
            ax.annotate(f'{int(height)}',
                        (bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2),
                        ha='center', va='center', color='white', size=9)

    # Save the plot as a PDF in the QC subfolder
    pdf_file = os.path.join(Balanced_dataset_folder, 'Value_Counts_Histogram.pdf')
    plt.savefig(pdf_file, bbox_inches='tight')
    print(f"Saved histogram to {pdf_file}")

    plt.show()

# Assuming dataset_df is your DataFrame and Results_Folder is defined
# result_df = count_values_by_condition_and_repeat(dataset_df, Results_Folder)

result_df = count_values_by_condition_and_repeat(dataset_df, Results_Folder)


## **3.2. Downsample your dataset to ensure that it is balanced**
--------

### Downsampling and Balancing Dataset

This section of the notebook is dedicated to addressing imbalances in the dataset, which is crucial for ensuring the accuracy and reliability of the analysis. The cell bellow will downsample the dataset to balance the number of tracks across different conditions and repeats. It allows for reproducibility by including a `random_seed` parameter, which is set to 42 by default but can be adjusted as needed.

All results from this section will be saved in the Balanced Dataset Directory created in your `Results_Folder`.




In [ ]:
import pandas as pd
import os

# @title ##Run this cell to downsample and balance your dataset

random_seed = 42  # @param {type: "number"}

if not os.path.exists(f"{Results_Folder}/Balanced_dataset"):
    os.makedirs(f"{Results_Folder}/Balanced_dataset")

def balance_dataset(df, condition_col='Condition', repeat_col='Repeat', random_seed=None):
    """
    Balances the dataset by downsampling rows for each condition and repeat combination.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    condition_col (str): The name of the column representing the condition.
    repeat_col (str): The name of the column representing the repeat.
    random_seed (int, optional): The seed for the random number generator. Default is None.

    Returns:
    pandas.DataFrame: A new DataFrame with balanced row counts.
    """
    # Group by condition and repeat, and find the minimum row count for any group
    min_row_count = df.groupby([condition_col, repeat_col]).size().min()

    # Function to sample min_row_count rows from each group
    def sample_rows(group):
        return group.sample(n=min_row_count, random_state=random_seed)

    # Apply sampling to each group and concatenate the results
    balanced_df = df.groupby([condition_col, repeat_col], group_keys=False).apply(sample_rows)

    return balanced_df

# Apply the function to your dataset
balanced_dataset_df = balance_dataset(dataset_df, random_seed=random_seed)


## **3.3. Check if the downsampling has affected data distribution**
--------

This section of the notebook generates a heatmap visualizing the Kolmogorov-Smirnov (KS) p-values for each numerical column in the dataset, comparing the distributions before and after downsampling. This heatmap serves as a tool for assessing the impact of downsampling on data quality, guiding decisions on whether the downsampled dataset is suitable for further analysis.

#### Purpose of the Heatmap
- **KS Test:** The KS test is used to determine if two samples are drawn from the same distribution. In this context, it compares the distribution of each numerical column in the original dataset (`merged_tracks_df`) with its counterpart in the downsampled dataset (`balanced_merged_tracks_df`).
- **P-Value Interpretation:** The p-value indicates the probability that the two samples come from the same distribution. A higher p-value suggests a greater likelihood that the distributions are similar.

#### Interpreting the Heatmap
- **Color Coding:** The heatmap uses a color gradient (from viridis) to represent the range of p-values. Darker colors indicate higher p-values.
- **P-Value Thresholds:**
  - **High P-Values (Lighter Areas):** Indicate that the downsampling process likely did not significantly alter the distribution of that numerical column for the specific condition-repeat group.
  - **Low P-Values (Darker Areas):** Suggest that the downsampling process may have affected the distribution significantly.
- **Varying P-Values:** Variations in color across different columns and rows help identify which specific numerical columns and condition-repeat groups are most affected by the downsampling.




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# @title ##Check if your downsampling has affected your data distribution

def calculate_ks_p_value(df1, df2, column):
    """
    Calculate the KS p-value for a given column between two dataframes.

    Parameters:
    df1 (pandas.DataFrame): Original DataFrame.
    df2 (pandas.DataFrame): DataFrame after downsampling.
    column (str): Column name to compare.

    Returns:
    float: KS p-value.
    """
    return ks_2samp(df1[column].dropna(), df2[column].dropna())[1]

# Identify numerical columns
numerical_columns = dataset_df.select_dtypes(include=['int64', 'float64']).columns

# Initialize a DataFrame to store KS p-values
ks_p_values = pd.DataFrame(columns=numerical_columns)

# Iterate over each group and numerical column
for group, group_df in dataset_df.groupby(['Condition', 'Repeat']):
    group_p_values = []
    balanced_group_df = balanced_dataset_df[(balanced_dataset_df['Condition'] == group[0]) & (balanced_dataset_df['Repeat'] == group[1])]
    for column in numerical_columns:
        p_value = calculate_ks_p_value(group_df, balanced_group_df, column)
        group_p_values.append(p_value)
    ks_p_values.loc[f'Condition: {group[0]}, Repeat: {group[1]}'] = group_p_values

# Maximum number of columns per heatmap
max_columns_per_heatmap = 20

# Total number of columns
total_columns = len(ks_p_values.columns)

# Calculate the number of heatmaps needed
num_heatmaps = -(-total_columns // max_columns_per_heatmap)  # Ceiling division

# File path for the PDF
pdf_filepath = Results_Folder+'/Balanced_dataset/p-Value Heatmap.pdf'

# Create a PDF file
with PdfPages(pdf_filepath) as pdf:
    # Loop through each subset of columns and create a heatmap
    for i in range(num_heatmaps):
        start_col = i * max_columns_per_heatmap
        end_col = min(start_col + max_columns_per_heatmap, total_columns)

        # Subset of columns for this heatmap
        subset_columns = ks_p_values.columns[start_col:end_col]

        # Create the heatmap for the subset of columns
        plt.figure(figsize=(12, 8))
        sns.heatmap(ks_p_values[subset_columns], cmap='viridis', vmax=0.5, vmin=0)
        plt.title(f'Kolmogorov-Smirnov P-Value Heatmap (Columns {start_col+1} to {end_col})')
        plt.xlabel('Numerical Columns')
        plt.ylabel('Condition-Repeat Groups')
        plt.tight_layout()

        # Save the current figure to the PDF
        pdf.savefig()
        plt.show()
        plt.close()

print(f"Saved all heatmaps to {pdf_filepath}")

# Save the p-values to a CSV file
ks_p_values.to_csv(Results_Folder + '/Balanced_dataset/ks_p_values.csv')
print("Saved KS p-values to ks_p_values.csv")


## **3.4. Plot your balanced dataset**
--------

In [ ]:
# @title ##Plot track parameters (balanced dataset)

import ipywidgets as widgets
from ipywidgets import Layout, VBox, Button, Accordion, SelectMultiple, IntText
import pandas as pd
import os
from matplotlib.ticker import FixedLocator


# Parameters to adapt in function of the notebook section
base_folder = f"{Results_Folder}/Balanced_dataset/track_parameters_plots"
Conditions = 'Condition'
df_to_plot = balanced_dataset_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def get_selectable_columns(df):
    # Exclude certain columns from being plotted
    exclude_cols = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']
    # Select only numerical columns
    return [col for col in df.columns if (df[col].dtype.kind in 'biufc') and (col not in exclude_cols)]


def display_variable_checkboxes(selectable_columns):
    # Create checkboxes for selectable columns
    variable_checkboxes = [widgets.Checkbox(value=False, description=col) for col in selectable_columns]

    # Display checkboxes in the notebook
    display(widgets.VBox([
        widgets.Label('Variables to Plot:'),
        widgets.GridBox(variable_checkboxes, layout=widgets.Layout(grid_template_columns="repeat(%d, 300px)" % 3)),
    ]))
    return variable_checkboxes

def create_condition_selector(df, column_name):
    conditions = df[column_name].unique()
    condition_selector = SelectMultiple(
        options=conditions,
        description='Conditions:',
        disabled=False,
        layout=Layout(width='100%')  # Adjusting the layout width
    )
    return condition_selector

def display_condition_selection(df, column_name):
    condition_selector = create_condition_selector(df, column_name)

    condition_accordion = Accordion(children=[VBox([condition_selector])])
    condition_accordion.set_title(0, 'Select Conditions')
    display(condition_accordion)
    return condition_selector


def plot_selected_vars(button, variable_checkboxes, df, Conditions, Results_Folder, condition_selector):

    plt.clf()  # Clear the current figure before creating a new plot
    print("Plotting in progress...")

  # Get selected variables
    variables_to_plot = [box.description for box in variable_checkboxes if box.value]
    n_plots = len(variables_to_plot)

    if n_plots == 0:
        print("No variables selected for plotting")
        return

  # Get selected conditions
    selected_conditions = condition_selector.value
    n_selected_conditions = len(selected_conditions)

    if n_selected_conditions == 0:
        print("No conditions selected for plotting")
        return

# Use only selected and ordered conditions
    filtered_df = df[df[Conditions].isin(selected_conditions)].copy()

# Initialize matrices to store effect sizes and p-values for each variable
    effect_size_matrices = {}
    p_value_matrices = {}
    bonferroni_matrices = {}

    unique_conditions = filtered_df[Conditions].unique().tolist()
    num_comparisons = len(unique_conditions) * (len(unique_conditions) - 1) // 2
    alpha = 0.05
    corrected_alpha = alpha / num_comparisons
    n_iterations = 1000

# Loop through each variable to plot
    for var in variables_to_plot:

      pdf_pages = PdfPages(f"{Results_Folder}/pdf/{var}_Boxplots_and_Statistics.pdf")
      effect_size_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      p_value_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)
      bonferroni_matrix = pd.DataFrame(index=unique_conditions, columns=unique_conditions)

      for cond1, cond2 in itertools.combinations(unique_conditions, 2):
        group1 = df[df[Conditions] == cond1][var]
        group2 = df[df[Conditions] == cond2][var]

        original_d = abs(cohen_d(group1, group2))
        effect_size_matrix.loc[cond1, cond2] = original_d
        effect_size_matrix.loc[cond2, cond1] = original_d  # Mirroring

        count_extreme = 0
        for i in range(n_iterations):
            combined = pd.concat([group1, group2])
            shuffled = combined.sample(frac=1, replace=False).reset_index(drop=True)
            new_group1 = shuffled[:len(group1)]
            new_group2 = shuffled[len(group1):]

            new_d = abs(cohen_d(new_group1, new_group2))
            if np.abs(new_d) >= np.abs(original_d):
                count_extreme += 1

        p_value = count_extreme / n_iterations
        p_value_matrix.loc[cond1, cond2] = p_value
        p_value_matrix.loc[cond2, cond1] = p_value  # Mirroring

        # Apply Bonferroni correction
        bonferroni_corrected_p_value = min(p_value * num_comparisons, 1.0)
        bonferroni_matrix.loc[cond1, cond2] = bonferroni_corrected_p_value
        bonferroni_matrix.loc[cond2, cond1] = bonferroni_corrected_p_value  # Mirroring

      effect_size_matrices[var] = effect_size_matrix
      p_value_matrices[var] = p_value_matrix
      bonferroni_matrices[var] = bonferroni_matrix

    # Concatenate the three matrices side-by-side
      combined_df = pd.concat(
        [
            effect_size_matrices[var].rename(columns={col: f"{col} (Effect Size)" for col in effect_size_matrices[var].columns}),
            p_value_matrices[var].rename(columns={col: f"{col} (P-Value)" for col in p_value_matrices[var].columns}),
            bonferroni_matrices[var].rename(columns={col: f"{col} (Bonferroni-corrected P-Value)" for col in bonferroni_matrices[var].columns})
        ], axis=1
    )

    # Save the combined DataFrame to a CSV file
      combined_df.to_csv(f"{Results_Folder}/csv/{var}_statistics_combined.csv")

    # Create a new figure
      fig = plt.figure(figsize=(16, 10))

    # Create a gridspec for 2 rows and 4 columns
      gs = GridSpec(2, 3, height_ratios=[1.5, 1])

    # Create the ax for boxplot using the gridspec
      ax_box = fig.add_subplot(gs[0, :])

    # Extract the data for this variable
      data_for_var = df[[Conditions, var, 'Repeat' ]]

    # Save the data_for_var to a CSV for replotting
      data_for_var.to_csv(f"{Results_Folder}/csv/{var}_boxplot_data.csv", index=False)

    # Calculate the Interquartile Range (IQR) using the 25th and 75th percentiles
      Q1 = df[var].quantile(0.25)
      Q3 = df[var].quantile(0.75)
      IQR = Q3 - Q1

    # Define bounds for the outliers
      multiplier = 10
      lower_bound = Q1 - multiplier * IQR
      upper_bound = Q3 + multiplier * IQR

    # Plotting
      sns.boxplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, color='lightgray')  # Boxplot
      sns.stripplot(x=Conditions, y=var, data=filtered_df, ax=ax_box, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
      ax_box.set_ylim([max(min(filtered_df[var]), lower_bound), min(max(filtered_df[var]), upper_bound)])
      ax_box.set_title(f"{var}")
      ax_box.set_xlabel('Condition')
      ax_box.set_ylabel(var)
      tick_labels = ax_box.get_xticklabels()
      tick_locations = ax_box.get_xticks()
      ax_box.xaxis.set_major_locator(FixedLocator(tick_locations))
      ax_box.set_xticklabels(tick_labels, rotation=90)
      ax_box.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

    # Statistical Analyses and Heatmaps

    # Effect Size heatmap ax
      ax_d = fig.add_subplot(gs[1, 0])
      sns.heatmap(effect_size_matrices[var].fillna(0), annot=True, cmap="viridis", cbar=True, square=True, ax=ax_d, vmax=1)
      ax_d.set_title(f"Effect Size (Cohen's d) for {var}")

    # p-value heatmap ax
      ax_p = fig.add_subplot(gs[1, 1])
      sns.heatmap(p_value_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_p, vmax=0.1)
      ax_p.set_title(f"Randomization Test p-value for {var}")

    # Bonferroni corrected p-value heatmap ax
      ax_bonf = fig.add_subplot(gs[1, 2])
      sns.heatmap(bonferroni_matrices[var].fillna(1), annot=True, cmap="viridis_r", cbar=True, square=True, ax=ax_bonf, vmax=0.1)
      ax_bonf.set_title(f"Bonferroni-corrected p-value for {var}")

      plt.tight_layout()
      pdf_pages.savefig(fig)

    # Close the PDF
      pdf_pages.close()

condition_selector = display_condition_selection(df_to_plot, Conditions)
selectable_columns = get_selectable_columns(df_to_plot)
variable_checkboxes = display_variable_checkboxes(selectable_columns)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, variable_checkboxes, df_to_plot, Conditions, base_folder, condition_selector))
display(button)

In [ ]:
import pandas as pd
from prettytable import PrettyTable
import os

# @title ##Export the data summaries


# Assuming Results_Folder, dataset_df, and Conditions are defined
save_path = f"{Results_Folder}/Balanced_dataset/track_parameters_plots"
df_to_plot = balanced_dataset_df  # Assuming dataset_df is the DataFrame to work with

if not os.path.exists(save_path):
  os.makedirs(save_path)

def generate_display_and_save_statistics(df, columns, Conditions, save_path):
    """
    Generates, displays using prettytable, and saves as CSV the statistical summaries
    for selected columns of the DataFrame, grouped by the specified condition column.

    Parameters:
    - df: DataFrame to analyze.
    - columns: List of column names to generate statistics for.
    - Conditions: Column name to group by.
    - save_path: Directory path where CSV files will be saved.
    """
    # Ensure the save directory exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for var in columns:
        if var in df.columns:
            # Compute descriptive statistics and additional metrics
            grouped_stats = df.groupby(Conditions)[var].describe()
            variance = df.groupby(Conditions)[var].var().rename('variance')
            skewness = df.groupby(Conditions)[var].skew().rename('skewness')
            kurtosis = df.groupby(Conditions)[var].apply(pd.DataFrame.kurt).rename('kurtosis')

            # Concatenate all statistics into a single DataFrame
            all_stats = pd.concat([grouped_stats, variance, skewness, kurtosis], axis=1)

            # Save the summary to a CSV file
            csv_filename = f"{var}_summary.csv"
            all_stats.to_csv(os.path.join(save_path, csv_filename))
            print(f"Saved statistical summary for {var} to {csv_filename}")

            # Initialize PrettyTable, using the DataFrame's columns as table fields
            table = PrettyTable()
            table.field_names = ["Condition"] + list(all_stats.columns)

            # Populate the table with data
            for condition, row in all_stats.iterrows():
                table.add_row([condition] + row.tolist())

            # Set table alignment, style, etc.
            table.align = 'r'
            print(f"Statistical Summary for {var}:\n{table}")

generate_display_and_save_statistics(df_to_plot, selectable_columns, Conditions, save_path)


# **Part 4. Version log**
---
<font size = 4>While I strive to provide accurate and helpful information, please be aware that:
  - This notebook may contain bugs.
  - Features are currently limited and will be expanded in future releases.

<font size = 4>We encourage users to report any issues or suggestions for improvement. Please check the [repository](https://github.com/guijacquemet/CellTracksColab) regularly for updates and the latest version of this notebook.


<font size = 4>**Version 0.1**
This is the first release of this notebook.

---